In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization,Dropout,InputLayer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from seaborn import heatmap

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def RA_DA():
  raw_data=tfds.as_numpy(tfds.load('colorectal_histology',split=['train'],batch_size=-1,shuffle_files = True,as_supervised=True,))

  img = raw_data[0][0]
  label = raw_data[0][1]

  return img ,label

In [ ]:
img, label = RA_DA()

In [ ]:
print('''Raw Data
Images: {0}
Each image size is: {1} '''.format(img.shape[0],img.shape[1::]))
#0-no of images
#1::-dimensions

In [ ]:
tissue_types = {0 : "Tumor" ,1 : "Debris" ,2 : "Stroma" ,3 : "Lympho" , 4 : "complex" , 5 : "Mucosa" ,6 : "Adipose"  ,7 : "Empty",  }
tissue_types

In [ ]:
for j in range(0,8,1):
    [idx] = np.where(label==j)
    plt.figure(figsize=(10,5))
    for i in range(10):
        plt.subplot(1,10,i+1)
        plt.imshow(img[idx[i]])
        plt.xticks([]), plt.yticks([])
        plt.title(tissue_types.get(label[idx[i]]))

In [ ]:
def split_to_train_and_test(X, y, test_ratio, rand_state = None):
    return train_test_split(X, y, test_size = test_ratio, random_state = rand_state)

def get_train_and_test_generator(train_images, train_labels, test_images, test_labels):


    data_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.1,
    shear_range=0.2,
    horizontal_flip=True)



    train_generator = data_gen_train.flow(x = train_images,y=train_labels,batch_size=32,shuffle=True)
    test_generator = data_gen_train.flow(x = test_images,y=test_labels,batch_size=32,shuffle=True)

    return train_generator, test_generator

In [ ]:
train_images,test_images, train_labels, test_labels = split_to_train_and_test(img, label, 0.2)
print(train_images.shape,test_labels.shape)

In [ ]:
values, counts = np.unique(test_labels, return_counts=True)
values

In [ ]:
counts

In [ ]:
def plot_generator_images(train_generator, all_label_dictionary):

  x_batch, y_batch = train_generator.next()

  plt.figure(figsize=(15,8))
  for i,x in enumerate(x_batch):
      plt.subplot(4,8,i+1)
      plt.imshow(x)
      plt.title(all_label_dictionary.get(y_batch[i]))
      plt.axis('off')

  print('x_batch.shape = ',x_batch.shape)
  print('y_batch.shape = ',y_batch.shape)

In [ ]:
train_generator, test_generator = get_train_and_test_generator(train_images, train_labels, test_images, test_labels)

In [ ]:
train_images[0]

In [ ]:
plot_generator_images(train_generator, tissue_types)

In [ ]:
plot_generator_images(test_generator, tissue_types)

In [ ]:
def plot_sample(X, y, index):
    plt.figure(figsize = (15,2))
    plt.imshow(X[index])
    plt.xlabel(tissue_types[y[index]])

In [ ]:
plot_sample(train_images, train_labels, 5)

In [ ]:
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

In [ ]:
model_1 = Sequential()

model_1.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding='same', input_shape=(150, 150,3)))
model_1.add(BatchNormalization())
model_1.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding='same'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(BatchNormalization())

model_1.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'))
model_1.add(BatchNormalization())
model_1.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding='same'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(BatchNormalization())

model_1.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(BatchNormalization())
model_1.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding='same'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))

model_1.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding='same'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.2))

model_1.add(Flatten())

model_1.add(Dense(128, activation='relu'))
model_1.add(Dropout(0.2))

model_1.add(Dense(1000, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(8, activation='softmax'))

model_1.summary()

In [ ]:
4000//32

In [ ]:
Adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.8, beta_2=0.999, epsilon=1e-07)

model_1.compile(optimizer=Adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_1 = model_1.fit(train_generator, steps_per_epoch=125, epochs=12,
                    validation_data=(test_generator))

In [ ]:
def plot_history_after_training(i_History):

    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    plt.plot(i_History.history['loss'],':or',label = "Train Data")
    plt.plot(i_History.history['val_loss'],'-og',label = "Test Data")
    plt.title('Loss',fontsize=14)
    plt.xlabel('Epochs',fontsize=14)
    plt.legend(loc = "upper left")
    plt.grid()

    plt.subplot(1,2,2)
    plt.plot(i_History.history['accuracy'],':ob',label = "Train Data")
    plt.plot(i_History.history['val_accuracy'],'-oc',label = "Test Data")
    plt.ylim([0, 1])
    plt.title('Accuracy',fontsize=12)
    plt.xlabel('Epochs',fontsize=12)
    plt.legend(loc = "upper left")
    plt.grid()

In [ ]:
plot_history_after_training(history_1)

In [ ]:
Adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.6, beta_2=0.999, epsilon=1e-07)

model_1.compile(optimizer=Adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


history_1_1 = model_1.fit(train_generator, steps_per_epoch=125, epochs=20,
                    validation_data=(test_generator))

In [ ]:
plot_history_after_training(history_1_1)

In [ ]:
model_1.save('model_1.h5')

In [ ]:
def plot_confusion_matrix(model, class_names, test_img, test_labels, model_name):
  yhat_1_hot = model.predict(test_img)
  yhat = np.argmax(yhat_1_hot, axis=1)
  matrix = confusion_matrix(test_labels, yhat)
  confusion_normalized_matrix = matrix / matrix.sum(axis=1)

  plt.figure(figsize=(10,8))
  heatmap(confusion_normalized_matrix,xticklabels=class_names, yticklabels=class_names,cmap='Blues',annot=True, fmt='.2%')
  plt.xlabel('Predicted label', fontsize=18)
  plt.ylabel('True label', fontsize=18)
  plt.title(str(model_name),fontsize=20)
  plt.show()

def get_accuracy_and_loss(model, test_images, test_labels):
  return model.evaluate(test_images, test_labels, verbose=0)

In [ ]:
get_accuracy_and_loss(model_1,test_images,test_labels)

In [ ]:
plot_confusion_matrix(model_1,tissue_types.values(),test_images,test_labels,'Model 1')

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
label_pred=model_1.predict(test_images)
label_pred_classes=[np.argmax(element) for element in label_pred]
print("Classification Report:\n",classification_report(test_labels,label_pred_classes))

In [ ]:
len(test_images)


In [ ]:
test_images[999]

In [ ]:
plot_sample(train_images, test_labels, 500)

In [ ]:
score = get_accuracy_and_loss(model_1, test_images, test_labels)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
tissue_types[test_labels[0]]

In [ ]:
label_pred= model_1.predict(test_images).round(2)
label_pred

##Visualize Output


In [ ]:
plt.figure(figsize=(16,30))
j=1
for i in np.random.randint(0,1000,60):
  plt.subplot(10,6,j);j+=1
  plt.imshow(test_images[i],cmap='Greys')
  plt.title('Actual = {} / {} \nPredicted = {} / {}'.format(tissue_types[test_labels[i]],test_labels[i],tissue_types[np.argmax(label_pred[i])],np.argmax(label_pred[i])))
  plt.axis('off')

##Model 2 using Transfer learning


In [ ]:
model_2 = keras.Sequential()

vgg16 = tf.keras.applications.VGG16(input_shape=(150,150,3),
                                               include_top=False,
                                               weights='imagenet')
vgg16.summary()

In [ ]:
print('#layer \t layer-name \t trainable')
print('------ \t ---------- \t ---------')

for (i,layer) in enumerate(vgg16.layers):
    print(str(i),'\t', layer.__class__.__name__, '\t',layer.trainable)

vgg16.trainable = False
print("\n  Make all layers not trainable   \n\n")

for (i,layer) in enumerate(vgg16.layers):
    print(str(i),'\t', layer.__class__.__name__, '\t',layer.trainable)

In [ ]:

model_2.add(vgg16)


model_2.add(Flatten())
model_2.add(Dense(256, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(8, activation='softmax'))

model_2.summary()



In [ ]:
GD_RMSprop = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.7, epsilon=1e-07)

model_2.compile(optimizer=GD_RMSprop,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_weights_freeze = model_2.fit(train_generator, steps_per_epoch=125, epochs=12, validation_data=(test_generator))

In [ ]:
plot_history_after_training(history_weights_freeze)

In [ ]:
for layer in model_2.layers:
    layer.trainable = True

model_2.summary()

In [ ]:
SGD_momentum = keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9)


model_2.compile(optimizer=SGD_momentum,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history_unfreeze = model_2.fit(train_generator, steps_per_epoch=125, epochs=20, validation_data=(test_generator))

In [ ]:
plot_history_after_training(history_unfreeze)

In [ ]:
model_2.save('model_2.h5')

In [ ]:
plt.figure(figsize=(25,7))

plt.subplot(1,2,1)

plt.plot(history_unfreeze.history['loss'],':or',label = "Train Data - Model 2 (VGG-like model)")
plt.plot(history_unfreeze.history['val_loss'],'-or',label = "Test Data - Model 2 (VGG-like model)")

plt.plot(history_1_1.history['loss'],':ob',label = "Train Data - Model 1")
plt.plot(history_1_1.history['val_loss'],'-ob',label = "Test Data - Model 1")

plt.title('Loss',fontsize=14)
plt.xlabel('Epochs',fontsize=14)
plt.legend(loc = "upper right")
plt.grid()

plt.subplot(1,2,2)

plt.plot(history_unfreeze.history['accuracy'],':or',label = "Train Data - Model 2 (VGG-like model)")
plt.plot(history_unfreeze.history['val_accuracy'],'-or',label = "Test Data - Model 2 (VGG-like model)")

plt.plot(history_1_1.history['accuracy'],':ob',label = "Train Data - Model 1")
plt.plot(history_1_1.history['val_accuracy'],'-ob',label = "Test Data - Model 1")

plt.ylim([0, 1])
plt.title('Accuracy',fontsize=12)
plt.xlabel('Epochs',fontsize=12)
plt.legend(loc = "lower right")
plt.grid()

In [ ]:
plt.figure(figsize=(10,7))

plt.plot(history_unfreeze.history['val_accuracy'],'-or',label = "Test Data - Model 2 (VGG-like model)")

plt.plot(history_1_1.history['val_accuracy'],'-ob',label = "Test Data - Model 1")



plt.ylim([0, 1])
plt.title('Accuracy',fontsize=12)
plt.xlabel('Epochs',fontsize=12)
plt.legend(loc = "lower right")
plt.grid()

In [ ]:
score = get_accuracy_and_loss(model_2, test_images, test_labels)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
plot_confusion_matrix(model_2,tissue_types.values(),test_images,test_labels,'Model 2 using Transfer Learning')

In [ ]:
model_2.save('model_2.h5')


In [ ]:
label_pred_2= model_2.predict(test_images).round(2)
label_pred_2

In [ ]:
plt.figure(figsize=(16,30))
j=1
for i in np.random.randint(0,1000,60):
  plt.subplot(10,6,j);j+=1
  plt.imshow(test_images[i],cmap='Greys')
  plt.title('Actual = {} / {} \nPredicted = {} / {}'.format(tissue_types[test_labels[i]],test_labels[i],tissue_types[np.argmax(label_pred_2[i])],np.argmax(label_pred_2[i])))
  plt.axis('off')

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
label_pred_2=model_2.predict(test_images)
label_pred_classes2=[np.argmax(element) for element in label_pred_2]
print("Classification Report:\n",classification_report(test_labels,label_pred_classes2))
